# One Stage Image Detection

> Faster RCNN还是有些慢
>
> 为什么要做两步？先预测感兴趣的区域再分类？能不能一步到位？
>
> 直接判定anchor属于哪个类别不可以吗？
>

生成anchor判定类别：SSD的思路

YOLO 速度精度高

# YOLO v1 (2015)

[论文地址](https://arxiv.org/pdf/1506.02640.pdf)

![1](1.PNG)

## 流程

1. 先在原图（448x448）划分7x7的网格，一个格子叫一个cell
2. 当且仅当物体中心落在一个cell上，那么这个cell负责预测这个物体。
    - 两个物体中心落在一个cell上，只会负责先来的（存在漏检情况）
    - 网格划分可以更细

输入： 448x448x3

输出： 7x7x（2x5+20）

每个cell预测2个bonding_box，这个bonding_box是回归出来的，是网络生成的

bonding_box有5个值: center_x, center_y, w, h, confidence

confidence(置信程度): P(是否包含感兴趣的物体) x IoU(gt和bbox的）

中心cell的P为1，非中心cell的P为0

20是分类的类别数。用confidence高的那个bbox

## Loss

![2](2.PNG)

设s是网格每条边数, 那么网格数就是s的平方

设B是bbox数量, 比如前文为2

$$\mathbf{1}_{i,j}^{obj}$$

1obji表示对象是否出现在单元格i中，1obij表示单元格i中的第j个边界框预测值负责该预测


### bbox位置 Loss

$$ \lambda_{coord}\sum_{i=0}^{s^2}\sum_{j=0}^{B}\mathbf{1}_{i,j}^{obj}[(x_{i}-\hat{x_{i}})^{2}+(y_{i}-\hat{y_{i}})^{2}]$$


\lambda_{coord} 一般取5,因为30个通道里位置的太少了

### bbox宽高 Loss

$$ \lambda_{coord}\sum_{i=0}^{s^2}\sum_{j=0}^{B}\mathbf{1}_{i,j}^{obj}[(\sqrt{w_{i}}-\sqrt{\hat{w_{i}}})^{2}+(\sqrt{h_{i}}-\sqrt{\hat{h_{i}}})^{2}]$$

用根号仍然显得很随意, 不能对大物体充分抑制, 对数更好
### 置信度 Loss
设C为置信度, C= P x iou
 
$$\sum_{i=0}^{s^2}\sum_{j=0}^{B}\mathbf{1}_{i,j}^{obj}(C_{i}-\hat{C_{i}})^2 + \lambda_{no\_obj}\sum_{i=0}^{s^2}\sum_{j=0}^{B}\mathbf{1}_{i,j}^{no\_obj}(C_{i}-\hat{C_{i}})^2$$

显然, 后面那个hat ci为0, 因为它不负责预测

\lambda_{no\_obj}取0.5,因为不负责预测的太多了

### 类别 Loss
$$\sum_{i=0}^{s^2}\mathbf{1}_{i}^{obj}\sum_{c\in classes}(p_i(c)-\hat{p_i}(c))^2$$

### 总Loss将这些Loss加起来即可

## 优点

1. 快
2. FP很低
3. 宁可漏检也不会误检

## 缺点

1. 一张图7x7的网格, 最多能预测多少个bbox? 
    - 7x7x2个bbox, 但是只能7x7x1个物体
2. 1个cell只能预测一个object
    - 对于小物体不太好
    - 对于拥挤的不太好
3. 使用了全连接, 最后变成7x7x30, 这里完全可以用卷积代替
4. 取根号并不最好
5. 没有BN
6. 训练集里没有的长宽比, 预测不好

这也导致yolo v1的precision很低, recall 不高, 



# YOLO v2 (2016)

[论文地址](https://arxiv.org/pdf/1612.08242.pdf)
## 从V1的改进

1. 加入BN

2. 高分辨率分类
    - 先用224x224训练
    - 用448x448进行finetune
    - 给任务的训练集做finetune
    - 7x7变成13x13

3. **用anchor**

作者认为faster-rcnn中人为指定9种anchor不合适

这里对训练集的gt数据用k-means聚类得到Anchor

这里计算的是框和框之间的距离

每个cell有5个anchor, 总共13x13x5=845个


4. 细粒度特征融合 Fine-Grained features

> CNN提取语义信息(高级特征), 会同时损失像素信息(低级特征)
>
> CNN也同时损失了一些位置信息

我们想把低级特征和高级特征融合在一起

可以做一个分支, 把低级和高级的直接相加

但是有尺寸问题

作者提出一个新的层 reorg

把一个大图变成多个小图, 再和小图相加

这样能更好的用低级特征

5. 多尺度训练 Multi-Scale Training

多个不同分辨率进行训练, 去除了FC层, 就可以接受任意尺寸输入了

## anchor计算

 
5个anchor, 每个anchor有w和h, 共10个值

WH为整个图片的宽高

以a1为例:

$$a_{1_{w}}=\frac{a_{w_{ori}}}{W} \times 13$$
$$a_{1_{h}}=\frac{a_{h_{ori}}}{H} \times 13$$

原始有真正的bbox,(gt), 4个值
$$[x_{o},y_{o},w_{o},h_{o}]$$
归一化:
$$[x_{r},y_{r},w_{r},h_{r}]= [x_{o}/W,y_{o}/H,w_{o}/W,h_{o}/H]$$
归一化之后, 他们都介于0,1之间

乘以13:
$$[x,y,w,h]=[13x_{o}/W,13y_{o},13w_{o}/W,13h_{o}/H]$$

当前格化, 也就是减去索引, 也就是减去整数部分:
$$[x_{f},y_{f},w_{f},h_{f}] = [x-\left \lfloor  {x}\right \rfloor,y-\left \lfloor {y} \right \rfloor,\log{\frac{w}{anchors[0]}},\log{\frac{h}{anchors[1]}}]$$

此时, x_f,y_f坐标就是相对于某个格子左上角的相对坐标,

而不是相对整张图左上角的

# YOLO v3 (2018)
[原文地址](https://pjreddie.com/media/files/papers/YOLOv3.pdf)

借鉴了ResNet

参考了FPN, 特征金字塔网络

多尺度特征网络

3个尺度的cell划分:
- 13x13
- 26x26
- 52x52

每个cell有3个anchor, 总9个


分类是80类, 多标签, 

多个二分类 

用的logistic

3个尺度每个都会输出

之前是一个cell预测一个, 输出量:2x(4+1)+20

现在是一个anchor预测一个, 输出量:

13x13x3x(4+1+80) + 26x26x3x(4+1+80) + 52x52x3x(4+1+80)

注意把80放在括号里了

## FPN Feature Pyramid Network

[论文地址](https://arxiv.org/pdf/1612.03144.pdf)
### 简述:

既要参考高级的语义特征,又想参考低级的像素特征

图片卷积不断变小, 变小的可以拿出来, 放大(插值/转置卷积), 与原来的图elementwise相加

![31](31.PNG)

注意,它每一层都会输出一个结果

这样, 它能输出多个不同尺度的feature map

![33](33.PNG)

其中3x3是为了消除aliasing effect

把大的anchor对应到小的特征图上, 把小的对应到大的特征图上

![35](35.PNG)

# Retina Net 2018

[论文地址](https://arxiv.org/pdf/1708.02002.pdf)

作者认为one stage精度不行, 不在于没做RPN, 而是因为Loss不行

CEloss, 只是对正负样本, 不同类别物体做了分类

没有考虑分类难易程度的不均衡, 正负样本分布的不均衡 

某个样本多, 产生的Loss就多, 训练的模型就朝着那个样本的方向偏
## Focal Loss

产生Loss多的, 权重小

交叉熵中P接近01的容易判断, 而P=0.5的就是难的样本

$$Loss: \;\sum-\alpha(1-P)^{\alpha} \log P$$


# SSD 2015

加了anchor的多尺度预测

anchor直接预测类别

都是来自不同尺度特征图的

anchor free的网络是一种趋势

# mAP mean average precision

判断一个物体框对于否, 需要一个IoU阈值

比如:

超过0.7, 就是TP

小于0.3 ,就是FN

每个类别都有混淆矩阵, 不同的阈值对应了混淆矩阵里4个值

当某一类别固定, 对不同的阈值画曲线, 叫PR curve

Precision:你预测为真的有多少是真的真?

$$ Precision = \frac{TP}{TP+FP}$$


Recall: 对于所有真实为真的, 你预测对了多少?

$$ Recall = \frac{TP}{TP+FN}$$

## PR曲线 Precision - Recall Curve

![50](50.PNG)


通过不同的模型得到这么一条曲线, 

如何选点: 一般要均衡 precision和 Recall,  要看一些指标, 比如: 
$$ F1\_score = \frac{2}{\frac{1}{Precision}+\frac{1}{Recall}}$$

面积auc应该越大越好

不同的类别曲线不一样, 所有类别的曲线下的面积, 平均

就是mAP

# YOLO v3 代码

[重写的版本: git地址](github.com/reigngt09/yolov3workflow)

YOLO v3 跨平台用pytorch重写, 会变慢


# TODO

v1的loss不明白

v2的归一化, 那个anchors啥意思

P2-P6啥意思